In [1]:
import numpy as np 
import pandas as pd 
import requests
from tqdm.notebook import tqdm_notebook
tqdm_notebook.pandas()
from datetime import datetime

C:\Users\kfuan\Anaconda3\envs\dl_fastai\lib\site-packages\tqdm\std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
stocks_list = pd.read_csv('../data/sp_500_stocks.csv')

In [3]:
stocks_list

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


In [4]:
base_url = "https://sandbox.iexapis.com/stable/"

In [5]:
from secrets import IEX_CLOUD_API_TOKEN

In [6]:
def chunks(input_list, n):
    for i in range(0, len(input_list), n):
        yield input_list[i : i+n]

In [7]:
batch_symbols = [name for name in chunks(stocks_list.Ticker.values, 100)]

In [8]:
print(batch_symbols[0])

['A' 'AAL' 'AAP' 'AAPL' 'ABBV' 'ABC' 'ABMD' 'ABT' 'ACN' 'ADBE' 'ADI' 'ADM'
 'ADP' 'ADSK' 'AEE' 'AEP' 'AES' 'AFL' 'AIG' 'AIV' 'AIZ' 'AJG' 'AKAM' 'ALB'
 'ALGN' 'ALK' 'ALL' 'ALLE' 'ALXN' 'AMAT' 'AMCR' 'AMD' 'AME' 'AMGN' 'AMP'
 'AMT' 'AMZN' 'ANET' 'ANSS' 'ANTM' 'AON' 'AOS' 'APA' 'APD' 'APH' 'APTV'
 'ARE' 'ATO' 'ATVI' 'AVB' 'AVGO' 'AVY' 'AWK' 'AXP' 'AZO' 'BA' 'BAC' 'BAX'
 'BBY' 'BDX' 'BEN' 'BF.B' 'BIIB' 'BIO' 'BK' 'BKNG' 'BKR' 'BLK' 'BLL' 'BMY'
 'BR' 'BRK.B' 'BSX' 'BWA' 'BXP' 'C' 'CAG' 'CAH' 'CARR' 'CAT' 'CB' 'CBOE'
 'CBRE' 'CCI' 'CCL' 'CDNS' 'CDW' 'CE' 'CERN' 'CF' 'CFG' 'CHD' 'CHRW'
 'CHTR' 'CI' 'CINF' 'CL' 'CLX' 'CMA' 'CMCSA']


In [9]:
batch_strings = []
for batch in batch_symbols:
    batch_strings.append(",".join(batch))

In [10]:
print(batch_strings[0])

A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA


# Simple P/E based 

In [11]:
quant_value_portfolio = stocks_list.copy()

In [12]:
col_names = ['latestPrice', 'peRatio']

In [13]:
quant_value_portfolio[col_names] = None

In [14]:
quant_value_portfolio

,Ticker,latestPrice,peRatio
0,A,None,None
1,AAL,None,None
2,AAP,None,None
3,AAPL,None,None
4,ABBV,None,None
...,...,...,...
500,YUM,None,None
501,ZBH,None,None
502,ZBRA,None,None
503,ZION,None,None


In [15]:
def batch_api_call(symbols):
    batch_url = base_url+f"stock/market/batch?symbols={symbols}&types=quote&filter=latestPrice,peRatio&token={IEX_CLOUD_API_TOKEN}"
    return requests.get(batch_url).json()

In [16]:
data = batch_api_call(batch_strings[-1])

In [17]:
data

{'YUM': {'quote': {'latestPrice': 103.66, 'peRatio': 31.39}},
 'ZBH': {'quote': {'latestPrice': 164.73, 'peRatio': -227.26}},
 'ZBRA': {'quote': {'latestPrice': 424.54, 'peRatio': 47.99}},
 'ZION': {'quote': {'latestPrice': 49.93, 'peRatio': 21.26}},
 'ZTS': {'quote': {'latestPrice': 162.76, 'peRatio': 46.84}}}

In [18]:
d0 = datetime.now()

start = 0
for batch in tqdm_notebook(batch_strings):
    data = batch_api_call(batch)
    # use pandas.apply to each row in each batch
    for col in col_names:
        quant_value_portfolio[col][start:start+len(data)] = quant_value_portfolio['Ticker'][start:start+len(data)].apply(lambda x: data[x]['quote'][col])
    
    start += len(data)
    
print(datetime.now() - d0)


0:00:06.224152


In [19]:
quant_value_portfolio

,Ticker,latestPrice,peRatio
0,A,125.13,55.12
1,AAL,18.1,-1.27
2,AAP,164.37,23.71
3,AAPL,138.936,38.68
4,ABBV,109.66,23.9
...,...,...,...
500,YUM,107.39,31.32
501,ZBH,162,-219.6
502,ZBRA,422.02,48.19
503,ZION,49.85,20.97


In [20]:
pe_is_positive = quant_value_portfolio['peRatio'] > 0
quant_value_portfolio = quant_value_portfolio[pe_is_positive]
quant_value_portfolio.sort_values(by='peRatio', inplace=True)
quant_value_portfolio.reset_index(drop=True, inplace=True)
quant_value_portfolio = quant_value_portfolio[:50]
display(quant_value_portfolio.head(10))
print(quant_value_portfolio.shape)

<ipython-input-20-16bb2c5d053d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  quant_value_portfolio.sort_values(by='peRatio', inplace=True)


,Ticker,latestPrice,peRatio
0,NRG,44.03,2.63
1,NLOK,21.63,4.22
2,AIV,4.84,4.79
3,UNM,24.72,5.31
4,BIO,616.58,5.4
5,AFL,47.42,7.3
6,ALL,108.51,7.7
7,CPB,49.15,8.21
8,KIM,17,8.62
9,EBAY,61.34,8.68


(50, 3)


In [21]:
def get_equal_position_size(n):
    global portfolio_size
    global position_size
    while True:
        try:
            portfolio_size = float(input('Enter the size of portfolio: '))
            break
        except:
            print('Invalid Input!')
            continue
    position_size = portfolio_size/n
    print(f'Porfolio size: {portfolio_size}\nPosition size: {position_size}')

In [22]:
get_equal_position_size(quant_value_portfolio.shape[0])

Enter the size of portfolio: dasld
Invalid Input!
Enter the size of portfolio: ogfd
Invalid Input!
Enter the size of portfolio: 4582$
Invalid Input!
Enter the size of portfolio: 7582502
Porfolio size: 7582502.0
Position size: 151650.04


In [23]:
print(f'port. size: {portfolio_size}\npos. size: {position_size}')

port. size: 7582502.0
pos. size: 151650.04


In [24]:
quant_value_portfolio['amount of shares to buy'] = np.floor(position_size/quant_value_portfolio['latestPrice'])

In [25]:
quant_value_portfolio.head(10)

,Ticker,latestPrice,peRatio,amount of shares to buy
0,NRG,44.03,2.63,3444
1,NLOK,21.63,4.22,7011
2,AIV,4.84,4.79,31332
3,UNM,24.72,5.31,6134
4,BIO,616.58,5.4,245
5,AFL,47.42,7.3,3198
6,ALL,108.51,7.7,1397
7,CPB,49.15,8.21,3085
8,KIM,17,8.62,8920
9,EBAY,61.34,8.68,2472


# More multiples based
<ol>
    <li>P/E</li>
    <li>P/B</li>
    <li>P/S</li>
    <li>EV/EBITDA</li>
    <li>EV/GP (gross profit)</li>
</ol>

In [26]:
multiples_ratio_portfolio = stocks_list.copy()

In [27]:
multiples_ratio_portfolio

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


In [28]:
col_quote = ['latestPrice', 'peRatio']
col_advanced_stats = ['priceToBook', 'priceToSales', 'enterpriseValue', 'EBITDA', 'grossProfit']

In [29]:
multiples_ratio_portfolio[col_quote] = None 
multiples_ratio_portfolio[col_advanced_stats] = None

In [30]:
multiples_ratio_portfolio.head()

,Ticker,latestPrice,peRatio,priceToBook,priceToSales,enterpriseValue,EBITDA,grossProfit
0,A,None,None,None,None,None,None,None
1,AAL,None,None,None,None,None,None,None
2,AAP,None,None,None,None,None,None,None
3,AAPL,None,None,None,None,None,None,None
4,ABBV,None,None,None,None,None,None,None


In [31]:
def batch_api_ratio(symbols):
    batch_url = (base_url+f"stock/market/batch?symbols={symbols}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}")
    return requests.get(batch_url).json()

In [32]:
batch_api_ratio(batch_strings[-1])

{'YUM': {'advanced-stats': {'beta': 0.9520982779588391,
   'totalCash': 1111209485,
   'currentDebt': 1567053669,
   'revenue': 5858416657,
   'grossProfit': 4369491443,
   'totalRevenue': 5718581968,
   'EBITDA': 1812123103,
   'revenuePerShare': 19.05,
   'revenuePerEmployee': 169476.03,
   'debtToEquity': -0.7767476291191573,
   'profitMargin': 0.1985818587920967,
   'enterpriseValue': 42048469312,
   'enterpriseValueToRevenue': 7.43,
   'priceToSales': 5.81,
   'priceToBook': -4.08,
   'forwardPERatio': 26.32079214430469,
   'pegRatio': -6.455975758659115,
   'peHigh': 11.52395031585058,
   'peLow': 6.286845829355523,
   'week52highDate': '2020-12-19',
   'week52lowDate': '2020-03-11',
   'putCallRatio': 0.8343151406412874,
   'companyName': 'Yum Brands Inc.',
   'marketcap': 31981578491,
   'week52high': 113.04,
   'week52low': 56.8,
   'week52change': 0.012095626020614427,
   'sharesOutstanding': 309062547,
   'float': None,
   'avg10Volume': 2052381,
   'avg30Volume': 1684166,
 

In [33]:
d0 = datetime.now()

start = 0
for batch in tqdm_notebook(batch_strings):
    data = batch_api_ratio(batch)
    
    for col in col_quote:
        multiples_ratio_portfolio[col][start:start+len(data)] =  \
        multiples_ratio_portfolio['Ticker'][start:start+len(data)].apply(lambda x: data[x]['quote'][col])
    
    for col in col_advanced_stats:
        multiples_ratio_portfolio[col][start:start+len(data)] =  \
        multiples_ratio_portfolio['Ticker'][start:start+len(data)].apply(lambda x: data[x]['advanced-stats'][col])
    
    start += len(data)

    print(f'Timed{datetime.now() - d0}')

Timed0:00:01.738767
Timed0:00:03.352250
Timed0:00:04.923391
Timed0:00:06.522961
Timed0:00:08.184899
Timed0:00:09.207928



In [34]:
multiples_ratio_portfolio

,Ticker,latestPrice,peRatio,priceToBook,priceToSales,enterpriseValue,EBITDA,grossProfit
0,A,127.02,54.53,8.06,7.21,4.07756e+10,1.20207e+09,2.96906e+09
1,AAL,17.8,-1.27,-2,0.448,3.66355e+10,-5.72463e+09,2.50112e+10
2,AAP,162.69,24.05,2.89,1.13,1.12223e+10,1.01436e+09,4.52498e+09
3,AAPL,137.749,38.06,35.6,8.2,2.36557e+12,8.57775e+10,1.18216e+11
4,ABBV,108.64,24.5,12.85,4.85,2.76515e+11,1.68523e+10,2.8728e+10
...,...,...,...,...,...,...,...,...
500,YUM,103.08,31.04,-4.09,5.69,41391308399,1881405256,4240832750
501,ZBH,163.62,-219.34,2.9,4.92,42139512153,1477042947,5077159469
502,ZBRA,410.07,48.01,11.53,5.15,23299202223,739186463,1961901169
503,ZION,48.77,21.43,1.12,2.71,8060676041,695074702,2634022708


In [35]:
multiples_ratio_portfolio['EV/EBITDA'] = multiples_ratio_portfolio.enterpriseValue / multiples_ratio_portfolio.EBITDA
multiples_ratio_portfolio['EV/GrossProfit'] = multiples_ratio_portfolio.enterpriseValue / multiples_ratio_portfolio.grossProfit

In [36]:
multiples_ratio_portfolio

,Ticker,latestPrice,peRatio,priceToBook,priceToSales,enterpriseValue,EBITDA,grossProfit,EV/EBITDA,EV/GrossProfit
0,A,127.02,54.53,8.06,7.21,4.07756e+10,1.20207e+09,2.96906e+09,33.9211,13.7335
1,AAL,17.8,-1.27,-2,0.448,3.66355e+10,-5.72463e+09,2.50112e+10,-6.39962,1.46476
2,AAP,162.69,24.05,2.89,1.13,1.12223e+10,1.01436e+09,4.52498e+09,11.0634,2.48007
3,AAPL,137.749,38.06,35.6,8.2,2.36557e+12,8.57775e+10,1.18216e+11,27.578,20.0106
4,ABBV,108.64,24.5,12.85,4.85,2.76515e+11,1.68523e+10,2.8728e+10,16.4082,9.6253
...,...,...,...,...,...,...,...,...,...,...
500,YUM,103.08,31.04,-4.09,5.69,41391308399,1881405256,4240832750,22.0002,9.76018
501,ZBH,163.62,-219.34,2.9,4.92,42139512153,1477042947,5077159469,28.5296,8.29982
502,ZBRA,410.07,48.01,11.53,5.15,23299202223,739186463,1961901169,31.5201,11.8758
503,ZION,48.77,21.43,1.12,2.71,8060676041,695074702,2634022708,11.5968,3.06022


In [37]:
multiples_ratio_portfolio.drop(columns=['enterpriseValue', 'EBITDA', 'grossProfit'], inplace=True)

In [38]:
multiples_ratio_portfolio.columns.values[2:]

array(['peRatio', 'priceToBook', 'priceToSales', 'EV/EBITDA',
       'EV/GrossProfit'], dtype=object)

In [39]:
for col in multiples_ratio_portfolio.columns.values[2:]:
    multiples_ratio_portfolio[col+" Percentiles"] = multiples_ratio_portfolio[col].rank(pct=True)

In [40]:
multiples_ratio_portfolio

,Ticker,latestPrice,peRatio,priceToBook,priceToSales,EV/EBITDA,EV/GrossProfit,peRatio Percentiles,priceToBook Percentiles,priceToSales Percentiles,EV/EBITDA Percentiles,EV/GrossProfit Percentiles
0,A,127.02,54.53,8.06,7.21,33.9211,13.7335,0.861789,0.772541,0.800205,0.877049,0.797131
1,AAL,17.8,-1.27,-2,0.448,-6.39962,1.46476,0.157520,0.051230,0.032787,0.030738,0.057377
2,AAP,162.69,24.05,2.89,1.13,11.0634,2.48007,0.469512,0.409836,0.154713,0.260246,0.114754
3,AAPL,137.749,38.06,35.6,8.2,27.578,20.0106,0.747967,0.961066,0.831967,0.819672,0.930328
4,ABBV,108.64,24.5,12.85,4.85,16.4082,9.6253,0.471545,0.848361,0.674180,0.510246,0.639344
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,103.08,31.04,-4.09,5.69,22.0002,9.76018,0.628049,0.047131,0.731557,0.698770,0.649590
501,ZBH,163.62,-219.34,2.9,4.92,28.5296,8.29982,0.024390,0.412910,0.680328,0.827869,0.561475
502,ZBRA,410.07,48.01,11.53,5.15,31.5201,11.8758,0.829268,0.829918,0.700820,0.864754,0.729508
503,ZION,48.77,21.43,1.12,2.71,11.5968,3.06022,0.418699,0.114754,0.437500,0.282787,0.157787


In [41]:
# robust value(RV) score = mean of those percentiles
multiples_ratio_portfolio['RV score'] = multiples_ratio_portfolio.filter(like='Percentiles').mean(axis=1)

In [42]:
multiples_ratio_portfolio

,Ticker,latestPrice,peRatio,priceToBook,priceToSales,EV/EBITDA,EV/GrossProfit,peRatio Percentiles,priceToBook Percentiles,priceToSales Percentiles,EV/EBITDA Percentiles,EV/GrossProfit Percentiles,RV score
0,A,127.02,54.53,8.06,7.21,33.9211,13.7335,0.861789,0.772541,0.800205,0.877049,0.797131,0.821743
1,AAL,17.8,-1.27,-2,0.448,-6.39962,1.46476,0.157520,0.051230,0.032787,0.030738,0.057377,0.065930
2,AAP,162.69,24.05,2.89,1.13,11.0634,2.48007,0.469512,0.409836,0.154713,0.260246,0.114754,0.281812
3,AAPL,137.749,38.06,35.6,8.2,27.578,20.0106,0.747967,0.961066,0.831967,0.819672,0.930328,0.858200
4,ABBV,108.64,24.5,12.85,4.85,16.4082,9.6253,0.471545,0.848361,0.674180,0.510246,0.639344,0.628735
...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,103.08,31.04,-4.09,5.69,22.0002,9.76018,0.628049,0.047131,0.731557,0.698770,0.649590,0.551020
501,ZBH,163.62,-219.34,2.9,4.92,28.5296,8.29982,0.024390,0.412910,0.680328,0.827869,0.561475,0.501394
502,ZBRA,410.07,48.01,11.53,5.15,31.5201,11.8758,0.829268,0.829918,0.700820,0.864754,0.729508,0.790854
503,ZION,48.77,21.43,1.12,2.71,11.5968,3.06022,0.418699,0.114754,0.437500,0.282787,0.157787,0.282305


In [43]:
multiples_ratio_portfolio.sort_values(by='RV score', inplace=True)
multiples_ratio_portfolio.reset_index(drop=True, inplace=True)
multiples_ratio_portfolio = multiples_ratio_portfolio[:50]
multiples_ratio_portfolio.head(10)

,Ticker,latestPrice,peRatio,priceToBook,priceToSales,EV/EBITDA,EV/GrossProfit,peRatio Percentiles,priceToBook Percentiles,priceToSales Percentiles,EV/EBITDA Percentiles,EV/GrossProfit Percentiles,RV score
0,UNM,25.12,5.5,0.4696,0.4356,2.67642,0.40798,0.174797,0.057377,0.030738,0.036885,0.004098,0.060779
1,AIG,41.42,-7.31,0.5622,0.7787,4.70545,0.777124,0.115854,0.061475,0.077869,0.045082,0.018443,0.063745
2,AAL,17.8,-1.27,-2,0.448,-6.39962,1.46476,0.157520,0.051230,0.032787,0.030738,0.057377,0.065930
3,MCK,194.82,-7.58,-64.87,0.1253,7.45415,2.78491,0.113821,0.010246,0.004098,0.090164,0.137295,0.071125
4,F,11.43,-284,1.35,0.3406,4.66145,2.32671,0.022358,0.168033,0.012295,0.043033,0.110656,0.071275
5,MET,54.1,8.66,0.6538,0.7239,4.81161,0.715566,0.186992,0.065574,0.065574,0.047131,0.012295,0.075513
6,ALL,109.96,7.8,1.33,0.7494,2.41517,0.72266,0.178862,0.159836,0.071721,0.034836,0.014344,0.091920
7,HIG,52.97,10.69,1.09,0.8873,4.28064,0.851303,0.209350,0.109631,0.092213,0.040984,0.024590,0.095354
8,FTI,11.51,-0.9,1.27,0.3937,3.52439,2.27375,0.160569,0.142418,0.026639,0.038934,0.108607,0.095433
9,BA,216.15,-10.52,-6.7,2.13,-14.7246,-29.1124,0.103659,0.043033,0.348361,0.022541,0.002049,0.103928


In [44]:
multiples_ratio_portfolio.shape

(50, 13)

In [45]:
get_equal_position_size(multiples_ratio_portfolio.shape[0])

Enter the size of portfolio: fsd,l
Invalid Input!
Enter the size of portfolio: 5821498
Porfolio size: 5821498.0
Position size: 116429.96


In [46]:
multiples_ratio_portfolio['Amount of shares to buy'] = np.floor(position_size/multiples_ratio_portfolio.latestPrice)

<ipython-input-46-925aa2456434>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  multiples_ratio_portfolio['Amount of shares to buy'] = np.floor(position_size/multiples_ratio_portfolio.latestPrice)


In [47]:
multiples_ratio_portfolio.head(10)

,Ticker,latestPrice,peRatio,priceToBook,priceToSales,EV/EBITDA,EV/GrossProfit,peRatio Percentiles,priceToBook Percentiles,priceToSales Percentiles,EV/EBITDA Percentiles,EV/GrossProfit Percentiles,RV score,Amount of shares to buy
0,UNM,25.12,5.5,0.4696,0.4356,2.67642,0.40798,0.174797,0.057377,0.030738,0.036885,0.004098,0.060779,4634
1,AIG,41.42,-7.31,0.5622,0.7787,4.70545,0.777124,0.115854,0.061475,0.077869,0.045082,0.018443,0.063745,2810
2,AAL,17.8,-1.27,-2,0.448,-6.39962,1.46476,0.157520,0.051230,0.032787,0.030738,0.057377,0.065930,6541
3,MCK,194.82,-7.58,-64.87,0.1253,7.45415,2.78491,0.113821,0.010246,0.004098,0.090164,0.137295,0.071125,597
4,F,11.43,-284,1.35,0.3406,4.66145,2.32671,0.022358,0.168033,0.012295,0.043033,0.110656,0.071275,10186
5,MET,54.1,8.66,0.6538,0.7239,4.81161,0.715566,0.186992,0.065574,0.065574,0.047131,0.012295,0.075513,2152
6,ALL,109.96,7.8,1.33,0.7494,2.41517,0.72266,0.178862,0.159836,0.071721,0.034836,0.014344,0.091920,1058
7,HIG,52.97,10.69,1.09,0.8873,4.28064,0.851303,0.209350,0.109631,0.092213,0.040984,0.024590,0.095354,2198
8,FTI,11.51,-0.9,1.27,0.3937,3.52439,2.27375,0.160569,0.142418,0.026639,0.038934,0.108607,0.095433,10115
9,BA,216.15,-10.52,-6.7,2.13,-14.7246,-29.1124,0.103659,0.043033,0.348361,0.022541,0.002049,0.103928,538


# save to excel

In [48]:
writer = pd.ExcelWriter(path='value_strategy_portfolio.xlsx', engine='xlsxwriter')

In [49]:
quant_value_portfolio.to_excel(writer, sheet_name='basic PE based', index=False)
multiples_ratio_portfolio.to_excel(writer, sheet_name='multiple ratios based', index=False)

In [50]:
writer.save()